In [2]:
import torch 

torch.manual_seed(0)

print(torch.get_rng_state().shape)

torch.Size([5056])


In [10]:
from torch.utils.data import TensorDataset, DataLoader

data = torch.rand(100, 3)
dataset = TensorDataset(data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for j in range(10):
    for i, batch in enumerate(dataloader):
        pass

    print(torch.get_rng_state().shape, torch.get_rng_state().tolist())

torch.Size([5056]) [0, 0, 0, 0, 0, 0, 0, 0, 33, 0, 0, 0, 1, 0, 0, 0, 80, 2, 0, 0, 0, 0, 0, 0, 145, 161, 13, 205, 0, 0, 0, 0, 190, 29, 186, 148, 0, 0, 0, 0, 71, 136, 197, 99, 0, 0, 0, 0, 233, 15, 15, 211, 0, 0, 0, 0, 238, 165, 173, 110, 0, 0, 0, 0, 81, 140, 49, 90, 0, 0, 0, 0, 209, 51, 61, 107, 0, 0, 0, 0, 158, 226, 137, 98, 0, 0, 0, 0, 148, 80, 108, 179, 0, 0, 0, 0, 89, 52, 20, 180, 0, 0, 0, 0, 227, 211, 40, 135, 0, 0, 0, 0, 104, 76, 55, 141, 0, 0, 0, 0, 173, 11, 140, 50, 0, 0, 0, 0, 206, 225, 164, 140, 0, 0, 0, 0, 219, 200, 225, 6, 0, 0, 0, 0, 68, 136, 23, 183, 0, 0, 0, 0, 192, 204, 3, 108, 0, 0, 0, 0, 186, 167, 164, 160, 0, 0, 0, 0, 169, 217, 249, 85, 0, 0, 0, 0, 45, 155, 7, 195, 0, 0, 0, 0, 156, 142, 30, 90, 0, 0, 0, 0, 206, 129, 213, 78, 0, 0, 0, 0, 136, 138, 34, 2, 0, 0, 0, 0, 202, 213, 212, 43, 0, 0, 0, 0, 238, 227, 213, 183, 0, 0, 0, 0, 106, 132, 38, 49, 0, 0, 0, 0, 96, 92, 164, 166, 0, 0, 0, 0, 254, 126, 74, 19, 0, 0, 0, 0, 156, 156, 59, 72, 0, 0, 0, 0, 175, 11, 200, 22, 0, 0, 